In [67]:
import pandas as pd
import numpy as np

In [68]:
redfin = pd.read_csv('../1_data_raw/redfin_market_tracker_LI.csv')
redfin.head()

,Unnamed: 0,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,1146,2017-08-01,2017-08-31,30,county,5,1996,f,"Suffolk County, NY",NaN,...,-0.018610,0.165706,-0.031015,-0.045004,0.094444,0.002420,-0.014083,"Nassau County, NY",35004.0,2023-10-29 14:25:50
1,2300,2018-02-01,2018-02-28,30,county,5,1996,f,"Suffolk County, NY",NaN,...,-0.144231,0.138462,-0.039621,0.030618,0.000000,-0.133333,-0.090909,"Nassau County, NY",35004.0,2023-10-29 14:25:50
2,2885,2021-10-01,2021-10-31,30,county,5,1996,f,"Suffolk County, NY",NaN,...,0.034615,0.148148,0.010217,-0.151852,0.071429,-0.261905,-0.011905,"Nassau County, NY",35004.0,2023-10-29 14:25:50
3,4326,2023-08-01,2023-08-31,30,county,5,1996,f,"Suffolk County, NY",NaN,...,-0.135338,0.052632,0.002632,-0.072368,0.600000,0.327273,0.336842,"Nassau County, NY",35004.0,2023-10-29 14:25:50
4,4380,2013-04-01,2013-04-30,30,county,5,1996,f,"Suffolk County, NY",NaN,...,-0.005694,0.178679,0.036528,-0.010062,0.148196,-0.005650,0.016120,"Nassau County, NY",35004.0,2023-10-29 14:25:50


In [69]:
redfin.info()
#print(list(redfin.columns.values))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1406 entries, 0 to 1405
Data columns (total 59 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      1406 non-null   int64  
 1   period_begin                    1406 non-null   object 
 2   period_end                      1406 non-null   object 
 3   period_duration                 1406 non-null   int64  
 4   region_type                     1406 non-null   object 
 5   region_type_id                  1406 non-null   int64  
 6   table_id                        1406 non-null   int64  
 7   is_seasonally_adjusted          1406 non-null   object 
 8   region                          1406 non-null   object 
 9   city                            0 non-null      float64
 10  state                           1406 non-null   object 
 11  state_code                      1406 non-null   object 
 12  property_type                   14

In [70]:
# Columns:
# ['Unnamed: 0', 'period_begin', 'period_end', 'period_duration', 'region_type', 'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region', 'city', 'state', 'state_code', 'property_type', 'property_type_id', 'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy', 'median_list_price', 'median_list_price_mom', 'median_list_price_yoy', 'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf', 'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold', 'homes_sold_mom', 'homes_sold_yoy', 'pending_sales', 'pending_sales_mom', 'pending_sales_yoy', 'new_listings', 'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom', 'inventory_yoy', 'months_of_supply', 'months_of_supply_mom', 'months_of_supply_yoy', 'median_dom', 'median_dom_mom', 'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom', 'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom', 'sold_above_list_yoy', 'price_drops', 'price_drops_mom', 'price_drops_yoy', 'off_market_in_two_weeks', 'off_market_in_two_weeks_mom', 'off_market_in_two_weeks_yoy', 'parent_metro_region', 'parent_metro_region_metro_code', 'last_updated']

Notes:  
- possible dependent variable: median sale price  
- fewer null values (will probably impute or fill with 0)  
- also has info on how fast houses leave the market (**)  

## Fernando Notes

Combine period_begin and period_end to just one column, period.  

Drop the following columns:

- Col 1 (ID)
- Col 2 and 3 (Period_begin and period_end, replace with period)
- period_duration
- region_type, region_type_id, and table_id
- is_seasonally_adjusted
- city, state, state_code

In [71]:
# replace period_begin with period
import datetime
redfin['period'] = pd.to_datetime(redfin['period_begin'])
redfin['timestamp'] = redfin['period'].apply(lambda x: x.timestamp())

# merge with interest rate csv data
interest = pd.read_csv('../data_raw/avg_interest_rates.csv')
interest['date'] = pd.to_datetime(interest['date']).dt.strftime('%Y-%m')
redfin['interest_rate'] = redfin['period'].apply(lambda x: interest[interest['date'] == x.strftime('%Y-%m')]['rate'].values[0])


In [72]:
manual_drop = ['Unnamed: 0', 'period_begin', 'period_end', 'period_duration', 'region_type', 'region_type_id', 'table_id', 'is_seasonally_adjusted', 'city', 'state', 'state_code', 'parent_metro_region', 'parent_metro_region_metro_code', 'last_updated']
redfin.drop(manual_drop, axis=1, inplace=True)
redfin.drop([col for col in redfin.columns if '_mom' in col or '_yoy' in col], axis=1, inplace=True)

In [73]:
# save
redfin.to_csv('../3_data_cleaned/redfin_market_tracker_LI_cleanup.csv', index=False)
redfin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1406 entries, 0 to 1405
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   region                   1406 non-null   object        
 1   property_type            1406 non-null   object        
 2   property_type_id         1406 non-null   int64         
 3   median_sale_price        1406 non-null   float64       
 4   median_list_price        1403 non-null   float64       
 5   median_ppsf              1395 non-null   float64       
 6   median_list_ppsf         1403 non-null   float64       
 7   homes_sold               1406 non-null   float64       
 8   pending_sales            1388 non-null   float64       
 9   new_listings             1403 non-null   float64       
 10  inventory                1406 non-null   float64       
 11  months_of_supply         1406 non-null   float64       
 12  median_dom               1406 non-